In [1]:
import cv2
import os
import numpy as np
from typing import Tuple, Dict, List, Optional
from sklearn.model_selection import train_test_split
import logging
import pickle
from datetime import datetime

class FaceRecognitionSystem:
    def __init__(self, model_dir: str, min_confidence: float = 65.0):
        """
        Initialize the face recognition system.

        Args:
            model_dir: Directory to store model and related files
            min_confidence: Minimum confidence threshold for recognition (0-100)
        """
        self.model_dir = model_dir
        self.min_confidence = min_confidence
        self.recognizer = cv2.face.LBPHFaceRecognizer_create()
        self.face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )
        self.label_map = {}

        # Set up logging
        logging.basicConfig(
            filename=os.path.join(model_dir, 'face_recognition.log'),
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )

    def preprocess_image(self, image: np.ndarray) -> np.ndarray:
        """
        Preprocess image for better face detection.

        Args:
            image: Input image

        Returns:
            Preprocessed image
        """
        # Convert to grayscale if needed
        if len(image.shape) == 3:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Apply histogram equalization for better contrast
        image = cv2.equalizeHist(image)

        # Apply noise reduction
        image = cv2.GaussianBlur(image, (5, 5), 0)

        return image

    def detect_face(self, image: np.ndarray) -> List[Tuple[int, int, int, int]]:
        """
        Detect faces in the image using cascade classifier.

        Args:
            image: Input image

        Returns:
            List of face coordinates (x, y, w, h)
        """
        faces = self.face_cascade.detectMultiScale(
            image,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30),
            flags=cv2.CASCADE_SCALE_IMAGE
        )
        return faces

    def train_model(self, dataset_path: str, test_size: float = 0.2) -> None:
        """
        Train the face recognition model.

        Args:
            dataset_path: Path to the dataset directory
            test_size: Proportion of dataset to use for testing
        """
        faces = []
        labels = []
        current_label = 0

        logging.info(f"Starting model training with dataset: {dataset_path}")

        # Process dataset
        for folder_name in os.listdir(dataset_path):
            folder_path = os.path.join(dataset_path, folder_name)
            if not os.path.isdir(folder_path):
                continue

            self.label_map[current_label] = folder_name
            logging.info(f"Processing person: {folder_name}")

            face_count = 0
            for filename in os.listdir(folder_path):
                img_path = os.path.join(folder_path, filename)
                try:
                    img = cv2.imread(img_path)
                    if img is None:
                        logging.warning(f"Failed to load image: {img_path}")
                        continue

                    # Preprocess image
                    processed_img = self.preprocess_image(img)

                    # Detect faces
                    detected_faces = self.detect_face(processed_img)

                    for (x, y, w, h) in detected_faces:
                        face = processed_img[y:y+h, x:x+w]
                        # Resize to ensure consistent size
                        face = cv2.resize(face, (160, 160))
                        faces.append(face)
                        labels.append(current_label)
                        face_count += 1

                except Exception as e:
                    logging.error(f"Error processing {img_path}: {str(e)}")
                    continue

            logging.info(f"Processed {face_count} faces for {folder_name}")
            current_label += 1

        if not faces:
            raise ValueError("No faces detected in the dataset!")

        # Convert to numpy arrays
        faces = np.array(faces)
        labels = np.array(labels)

        # Split dataset
        X_train, X_test, y_train, y_test = train_test_split(
            faces, labels, test_size=test_size, random_state=42
        )

        # Train the model
        self.recognizer.train(X_train, y_train)

        # Evaluate on test set
        correct = 0
        total = len(X_test)

        for face, true_label in zip(X_test, y_test):
            pred_label, confidence = self.recognizer.predict(face)
            if pred_label == true_label:
                correct += 1

        accuracy = (correct / total) * 100
        logging.info(f"Model accuracy on test set: {accuracy:.2f}%")

        # Save model and label map
        self.save_model()

    def save_model(self) -> None:
        """Save the trained model and label map."""
        os.makedirs(self.model_dir, exist_ok=True)

        model_path = os.path.join(self.model_dir, 'model.yml')
        self.recognizer.save(model_path)

        label_map_path = os.path.join(self.model_dir, 'label_map.pkl')
        with open(label_map_path, 'wb') as f:
            pickle.dump(self.label_map, f)

        logging.info(f"Model and label map saved to {self.model_dir}")

    def load_model(self) -> None:
        """Load the trained model and label map."""
        model_path = os.path.join(self.model_dir, 'model.yml')
        label_map_path = os.path.join(self.model_dir, 'label_map.pkl')

        if not os.path.exists(model_path) or not os.path.exists(label_map_path):
            raise FileNotFoundError("Model or label map not found!")

        self.recognizer.read(model_path)
        with open(label_map_path, 'rb') as f:
            self.label_map = pickle.load(f)

        logging.info("Model and label map loaded successfully")

    def recognize_face(self, image_path: str) -> Optional[Tuple[str, float]]:
        """
        Recognize a face in the given image.

        Args:
            image_path: Path to the image

        Returns:
            Tuple of (person_name, confidence) or None if no face detected
        """
        try:
            img = cv2.imread(image_path)
            if img is None:
                logging.error(f"Failed to load image: {image_path}")
                return None

            processed_img = self.preprocess_image(img)
            faces = self.detect_face(processed_img)

            if len(faces) == 0:
                logging.warning("No faces detected in the image")
                return None

            # Use the largest face if multiple detected
            largest_face = max(faces, key=lambda rect: rect[2] * rect[3])
            x, y, w, h = largest_face

            face = processed_img[y:y+h, x:x+w]
            face = cv2.resize(face, (160, 160))

            label, confidence = self.recognizer.predict(face)

            if confidence > self.min_confidence:
                person_name = self.label_map.get(label, "Unknown")
                logging.info(f"Recognized {person_name} with confidence {confidence:.2f}")
                return person_name, confidence
            else:
                logging.info(f"Face detected but confidence too low: {confidence:.2f}")
                return "Unknown", confidence

        except Exception as e:
            logging.error(f"Error during recognition: {str(e)}")
            return None

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Initialize with the correct paths
face_system = FaceRecognitionSystem(
    model_dir='/content/drive/MyDrive/Attendance_Model_Mark2/face_recognition_model',
    min_confidence=65.0
)

# Use the verified path from the directory listing
verified_path = '/content/drive/MyDrive/Attendance_Training_Mark1'  # adjust this based on the actual path
face_system.train_model(verified_path)

In [8]:
# First load the trained model
face_system.load_model()

# Test with a single image
test_image_path = '/content/drive/MyDrive/Attendance_Testing_Mark1/23bcs044_test1.jpeg'  # adjust this path to your test image
result = face_system.recognize_face(test_image_path)

if result:
    person_name, confidence = result
    print(f"Person recognized: {person_name}")
    print(f"Confidence: {confidence:.2f}%")
else:
    print("No face detected or recognition failed")

# To test multiple images from your test folder:
test_folder = '/content/drive/MyDrive/Attendance_Testing_Mark1'  # adjust this path to your test folder

for filename in os.listdir(test_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(test_folder, filename)
        print(f"\nTesting image: {filename}")
        result = face_system.recognize_face(image_path)

        if result:
            person_name, confidence = result
            print(f"Person recognized: {person_name}")
            print(f"Confidence: {confidence:.2f}%")
        else:
            print("No face detected or recognition failed")

Person recognized: Dhruv Koli
Confidence: 73.83%

Testing image: 23bcs028_test.jpeg
Person recognized: Dhruv Koli
Confidence: 91.69%

Testing image: 23bcs044_test1.jpeg
Person recognized: Dhruv Koli
Confidence: 73.83%

Testing image: 23bcs044_test3.jpeg
Person recognized: Dhruv Koli
Confidence: 85.01%

Testing image: 23bcs044_test2.jpeg
Person recognized: Barghav Abhilash
Confidence: 115.71%
